In [1]:
from EAC import site
import os 
import pandas as pd
import re
import numpy as np
import sqlite3

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
training = site.training()

conn = sqlite3.connect('training.db')

In [18]:
conn.execute('DROP TABLE movements')
conn.commit()

OperationalError: no such table: movements

In [28]:
conn.execute(create_movement_table_q)

In [ ]:
create_meso_q = '''
CREATE TABLE upcomingTraining (
week int,
session char(20),
movement char(20),
method char(20),
sets int,
reps int,
weight float(4),
progNotes char(140),
targetReps int
)
'''

In [27]:
#each movement/lift is the pk
create_movement_table_q = '''
CREATE TABLE movements (
movement CHAR(40) PRIMARY KEY not null,
split CHAR(10),
actualMax int,
repMaxCalc float(4),
veloEquation CHAR(30),
repMax10 int,
repMax9 int,
repMax8 int,
repMax7 int,
repMax6 int,
repMax5 int,
repMax4 int,
repMax3 int,
repMax2 int,
kilos binary
);

'''

In [90]:
#methodID is a amalgamation of method and week has 8 columns
create_program_builder_q = '''
CREATE TABLE programBuilder (
methodID char(30) PRIMARY KEY not null,
method char(20),
week int,
sets int,
reps int,
weight float(4),
notes char(140),
targetReps int
);
'''

In [ ]:
#sessionID  = date_exercise
create_history_q = '''
CREATE TABLE history(
sessionID PRIMARY KEY not null,
sessionDate date,
session char(30),
movement CHAR(40),
method CHAR(20),
setsCompleted int,
repsCompleted int,
weightUsed int,
amrapTotal int,
newMax binary,
notes CHAR(200),
RepCalcM int);
'''

In [ ]:
#veloID = date_exercise_set_rep
create_velo_q='''
CREATE TABLE velo (
movement char(40),
sessionDate date,
weight_kg float(4),
weight_lb float(4),
set_ct int,
rep int,
ecc_con char(4),
MPV_ms float(4),
peak_velo float(4),
rom_cm float(2),
rom_in float(2),
watts int,
est_max_kg float(4),
est_max_lb float(4)


)
'''

In [ ]:
create_rpe_q = '''
CREATE TABLE rpe (
RPE float(1) PRIMARY KEY not null,
rep1 float(4),
rep2 float(4),
rep3 float(4),
rep4 float(4),
rep5 float(4),
rep6 float(4),
rep7 float(4),
rep8 float(4),
rep9 float(4),
rep10 float(4),
rep11 float(4),
rep12 float(4),
rep13 float(4),
rep14 float(4),
rep15 float(4),
rep16 float(4)
);'''

In [ ]:

conn.execute(create_program_builder_q)
# conn.execute(create_meso_q)
# conn.execute(create_rpe_q)
# conn.execute(create_history_q)
# conn.execute(create_movement_table_q)
# conn.execute(create_velo_q)
conn.commit()


### Data Loading

In [7]:
def create_meso_cycle(path,connection):
    df = pd.DataFrame(columns = ['movement','method','week','sets','reps','weight','notes','targetReps'])
    prgbld = pd.read_sql('''
    select * 
    from programBuilder
    ''',conn)
    mesoDF = pd.read_csv(path).fillna(0)
    v = df.columns[-5:].values
    r=0
    for sesh in mesoDF.session.unique():
        d = mesoDF[mesoDF.session == sesh]
        for movement in d.exercise.unique():
            endweek = d[d.exercise==movement].endWeek.values[0]
            mod = d[d.exercise==movement].modality.values[0]
            cyclelength = prgbld[prgbld.method==mod].week.max()
            setAdjust = d[d.exercise==movement].SetAdjust.values[0]
            for i in range(endweek):
                lkup = i+1 if cyclelength==endweek else (i+1) % cyclelength
                lkup = cyclelength if lkup == 0 else lkup
                l = [movement,mod,i+1]+prgbld[(prgbld.method==mod) & (prgbld.week==lkup)].filter(v).values.tolist()[0]
                df.loc[len(df)] = l
                df.at[r,'sets'] = df.at[r,'sets'] + setAdjust
                r+=1
    path = './initialData/meso_file.csv'
    df.to_csv(path)
    return df,path
def add_data(path,table,connection,insert=False):
    if insert:
        cols = pd.read_sql('select * from {} limit 1'.format(table),conn).columns
        qs = (len(cols)-1)*'?,'+'?'
        connection.execute('''INSERT into {t} values ({row})'''.format(t=table,row=qs),path)
        i = 1
    else:
        with open(path,'r') as d:
            for i,row in enumerate(d):
                if i==0:
                    continue
                else:
                    row = row.replace('\n','')
                    qs = (len(row.split(','))-1)*'?,'+'?'
                    connection.execute('''INSERT into {t} values ({row})'''.format(t=table,row=qs),row.split(','))
    
    connection.commit()
    print('{} has been updated with {} rows of data'.format(table,i))
    
def update_movements_table(training_path,conn):
    '''
    Will take a csv of the training file, will find differences and update the database
    '''
    training = pd.read_csv(training_path,skiprows=1)
    training.movement = training.movement.str.lower()
    df = training.filter(['movement']+[col for col in training.columns if col.find('wrk')>-1])
    df = df.dropna().set_index('movement')
    mvmts = '","'.join(df.index.values)
    mvmt = pd.read_sql('''select * from movements where movement in ("{}")'''.format('","'.join(mvmts)),conn)
    ct=0
    for lft in df.index.values:
        try:
            if lft not in mvmt.movement.values:
                print('Adding in {} into movements table'.format(lft))
                d = [lft] + [np.nan] * (len(mvmt.columns) - 1)
                add_data(d,'movements',conn,True)
                ct+=1
            elif df.loc[lft]['wrk_10'] != int(mvmt[mvmt.movement=='convo']['actualMax'].values[0]):
                conn.execute('''UPDATE movements
                                    SET actualMax = {0} where movement = {1}'''.format(df.loc[lft]['wrk_10'],lft))
                ct+=1
            conn.commit()
        except:
            print('error')
    print('updated movements table with {} edits'.format(ct))

In [8]:
def update_movements_table(training_path,conn):
    '''
    Will take a csv of the training file, will find differences and update the database
    '''
    training = pd.read_csv(training_path,skiprows=1)
    training.movement = training.movement.str.lower()
    df = training.filter(['movement']+[col for col in training.columns if col.find('wrk')>-1])
    df = df.dropna().set_index('movement')
    mvmts = '","'.join(df.index.values)
    mvmt = pd.read_sql('''select * from movements where movement in ("{}")'''.format('","'.join(mvmts)),conn)
    ct=0
    for lft in df.index.values:
        try:
            if lft not in mvmt.movement.values:
                print('Adding in {} into movements table'.format(lft))
                d = [lft] + [np.nan] * (len(mvmt.columns) - 1)
                add_data(d,'movements',conn,True)
                ct+=1
            elif df.loc[lft]['wrk_10'] != int(mvmt[mvmt.movement=='convo']['actualMax'].values[0]):
                conn.execute('''UPDATE movements
                                    SET actualMax = {0} where movement = {1}'''.format(df.loc[lft]['wrk_10'],lft))
                ct+=1
            conn.commit()
        except:
            print('error')
    print('updated movements table with {} edits'.format(ct))

## Process in the future is 
1. run update_movements_table()
2. create meso
3. load into our google sheet
4. when done, upload training file in update movements table

### next steps:
we have the setup to find if lifts only went up during the cycle - .is_monotonic_increasing
can grab the new max from wrk10
\

In [77]:
import os
dataPath = './InitialData/'

In [84]:
pgFile = 'MesoLoad_06022023 - ProgramBuilder.csv'
moveFile = 'MesoLoad_06022023 - lifts.csv'
rpeFile='MesoLoad_06022023 - RPE.csv'
trainingFile = 'MesoLoad_052522024 - MesoLoad_18_PPL.csv'

In [127]:
conn.execute('DROP TABLE programBuilder')
conn.execute(create_program_builder_q)
conn.commit()

In [45]:
add_data(dataPath + moveFile,'movements',conn)

'movements has been updated with 78 rows of data'

In [128]:
add_data(dataPath + pgFile,'programBuilder',conn)

programBuilder has been updated with 1 rows of data
programBuilder has been updated with 2 rows of data
programBuilder has been updated with 3 rows of data
programBuilder has been updated with 4 rows of data
programBuilder has been updated with 5 rows of data
programBuilder has been updated with 6 rows of data
programBuilder has been updated with 7 rows of data
programBuilder has been updated with 8 rows of data
programBuilder has been updated with 9 rows of data
programBuilder has been updated with 10 rows of data
programBuilder has been updated with 11 rows of data
programBuilder has been updated with 12 rows of data
programBuilder has been updated with 13 rows of data
programBuilder has been updated with 14 rows of data
programBuilder has been updated with 15 rows of data
programBuilder has been updated with 16 rows of data
programBuilder has been updated with 17 rows of data
programBuilder has been updated with 18 rows of data
programBuilder has been updated with 19 rows of data
pr

In [131]:
meso,mpath = create_meso_cycle(dataPath+trainingFile,conn)
#add_data(mpath,'upcomingTraining',conn)

In [ ]:
db.create_meso_cycle(dataPath+trainingFile)

In [16]:
_,path = max([(os.stat(f)[-1],f.path) for f in os.scandir('./InitialData/') if re.search('PPL.csv$',f.name)!=None])

In [ ]:
pd.read_sql('select * from 

In [9]:

_,path,prgname = max([(os.stat(f)[-1],f.path,f.name) for f in os.scandir('./InitialData/') if re.search('PPL.csv$',f.name)!=None])
_,history,histname = max([(os.stat(f)[-1],f.path,f.name) for f in os.scandir('./InitialData/') if re.search('TrainingCopy.csv$',f.name)!=None])
print('Have {} found for upcoming training and {} loading history'.format(prgname,histname))
update_movements_table(history,conn)
create_meso_cycle(path,conn)
print('All updates are complete get new file')

Have MesoLoad_052522024 - MesoLoad_18_PPL.csv found for upcoming training and training - TrainingCopy.csv loading history
Adding in slingshot into movements table
error
Adding in bandbell incline into movements table
error
Adding in hatfield into movements table
error
Adding in tsunami squat into movements table
error
Adding in trap into movements table
error
Adding in meadows into movements table
error
Adding in pin press  into movements table
error
Adding in standing mil into movements table
error
Adding in tempo into movements table
error
Adding in gm into movements table
error
Adding in pendlay row into movements table
error
Adding in seal row into movements table
error
updated movements table with 0 edits


IndexError: list index out of range

In [26]:
df = pd.DataFrame(columns = ['movement','method','week','sets','reps','weight','notes','targetReps'])
prgbld = pd.read_sql('''
select methodID, lower(method) as method, week, sets,reps,weight,notes,targetReps
from programBuilder
''',conn)
mesoDF = pd.read_csv(path).fillna(0)

v = df.columns[-5:].values
r=0
for sesh in mesoDF.session.unique():
    d = mesoDF[mesoDF.session == sesh]
    for movement in d.exercise.unique():
        endweek = d[d.exercise==movement].endWeek.values[0]
        mod = d[d.exercise==movement].modality.values[0]
        cyclelength = prgbld[prgbld.method==mod].week.max()
        setAdjust = d[d.exercise==movement].SetAdjust.values[0]
        for i in range(endweek):
            lkup = i+1 if cyclelength==endweek else (i+1) % cyclelength
            lkup = cyclelength if lkup == 0 else lkup
            l = [movement,mod,i+1]+prgbld[(prgbld.method==mod) & (prgbld.week==lkup)].filter(v).values.tolist()[0]
            df.loc[len(df)] = l
            df.at[r,'sets'] = df.at[r,'sets'] + setAdjust
            r+=1

In [21]:
[movement,mod,i]

['squat', 'BandTech', 0]

In [25]:
mesoDF

,endWeek,session,ordering,exercise,modality,SetAdjust
0,9,Push1,1,Bench,9supp,0.0
1,9,Push1,2,Bandbell Incline,9hyp,0.0
2,9,Legs1,1,squat,BandTech,-3.0
3,9,Legs1,2,Front Squat,9str,-1.0
4,9,Legs1,3,Tsunami Squat,9supp,0.0
5,9,Pull1,1,Sumo,BandTech,-3.0
6,9,Pull1,2,Trap,9supp,-1.0
7,9,Pull1,3,seal row,9hyp,1.0
8,9,Push2,2,Standing Mil,9hyp,0.0
9,9,Push2,1,Pin Press,9supp,0.0
